In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib as mpl
from matplotlib.colors import BoundaryNorm
from matplotlib.colors import to_rgba
import matplotlib.cm as cm
import xarray as xr
import numpy as np
from pathlib import Path
from matplotlib import cm, colors


# === Define paths ===
path_q_gis_in = '../data/camels/CAMELScl_boundaries_epsg4326.gpkg'
path_gw_results = '../analysis/4_GW_Q_memory/csv/gw_memory.csv'
path_q_results = '../analysis/4_GW_Q_memory/csv/q_memory.csv'
path_netcdf = '../data/other/elevation_srtm_v4.1_Chile_2000_1200m_epsg4326.nc'

# === Load the region shapefile ===
path_shp = '../data/other/cl_regiones_geo/cl_regiones_geo.shp'
regions_gdf = gpd.read_file(path_shp)

# === Load the CSV files ===
gw_data = pd.read_csv(path_gw_results)
gw_attrib = pd.read_csv('../data/cr2sub/cr2sub_v1.1_attributes.csv')
q_data = pd.read_csv(path_q_results)
q_attrib = pd.read_csv('../data/camels/catchment_attributes.csv')

# === Load the GeoPackage files ===
gwl_gpkg = gpd.GeoDataFrame(
    gw_attrib.rename(columns={'dga_well_code': 'well_id'}),
    geometry=gpd.points_from_xy(gw_attrib['dga_well_lon'], gw_attrib['dga_well_lat']),
    crs='EPSG:4326'
)
q_gpkg = gpd.read_file(path_q_gis_in)

# === Load the NetCDF file ===
ds = xr.open_dataset(path_netcdf)
print(ds)
elevation = ds['elevation_srtm_v4.1_Chile_2000_1200m_epsg4326']


# === Strip spaces and ensure strings ===
gwl_gpkg['well_id'] = gwl_gpkg['well_id'].astype(str).str.strip()
gw_data['well_id'] = gw_data['well_id'].astype(str).str.strip()

q_gpkg['gauge_id'] = q_gpkg['gauge_id'].astype(str).str.strip()
q_data['gauge_id'] = q_data['gauge_id'].astype(str).str.strip()

# === Pad IDs with leading zeros to match formats (if needed) ===
max_length_gw = max(gwl_gpkg['well_id'].str.len().max(), gw_data['well_id'].str.len().max())
max_length_q = max(q_gpkg['gauge_id'].str.len().max(), q_data['gauge_id'].str.len().max())

gwl_gpkg['well_id'] = gwl_gpkg['well_id'].str.zfill(max_length_gw)
gw_data['well_id'] = gw_data['well_id'].str.zfill(max_length_gw)

q_gpkg['gauge_id'] = q_gpkg['gauge_id'].str.zfill(max_length_q)
q_data['gauge_id'] = q_data['gauge_id'].str.zfill(max_length_q)

# === Merge CSV with GeoPackage ===
gwl_merged = gwl_gpkg.merge(gw_data, left_on='well_id', right_on='well_id', how='left')
q_merged = q_gpkg.merge(q_data, left_on='gauge_id', right_on='gauge_id', how='left')

# === Save the new GeoPackages ===
gwl_gpkg = gwl_merged
q_gpkg = q_merged

# === Fixing Multi-part geometries ===
if q_gpkg.geom_type.isin(['MultiPolygon', 'MultiLineString']).any():
    q_gpkg = q_gpkg.explode(index_parts=False)

if gwl_gpkg.geom_type.isin(['MultiPolygon', 'MultiLineString']).any():
    gwl_gpkg = gwl_gpkg.explode(index_parts=False)

# === Remove any invalid geometries ===
q_gpkg = q_gpkg[q_gpkg.is_valid]
gwl_gpkg = gwl_gpkg[gwl_gpkg.is_valid]


# === bins for the map ===

# Step 1: Clean memory values
q_vals_clean = q_gpkg['q_memory'].dropna()
gw_vals_clean = gwl_gpkg['gw_memory'].dropna()

# Step 2: Define number of quantile bins for each
n_bins_q = 8
n_bins_gw = 12

q_bins = np.quantile(q_vals_clean, np.linspace(0, 1, n_bins_q + 1))
gw_bins = np.quantile(gw_vals_clean, np.linspace(0, 1, n_bins_gw + 1))

# Step 4: Merge, round, and ensure uniqueness
bins = np.round(np.unique(np.concatenate([q_bins, gw_bins])), 3)
bins[0] = 0  # ensure first bin starts at 0
cmap_list = cm.get_cmap('inferno')(np.linspace(1, 0, len(bins) - 1))
cmap = ListedColormap(cmap_list)
norm = BoundaryNorm(bins, cmap.N)
boundaries = bins

labels = [str(b) for b in bins[:-1]]  # Exclude the last bin
tick_positions = bins[:-1]            # Exclude the last bin


# === Map hydro memory values to bin indices using (a, b] logic ===
q_gpkg['bin_idx'] = pd.cut(
    q_gpkg['q_memory'],
    bins=bins,
    right=True,            # (a, b] intervals
    include_lowest=False,
    labels=False
)

gwl_gpkg['bin_idx'] = pd.cut(
    gwl_gpkg['gw_memory'],
    bins=bins,
    right=True,
    include_lowest=False,
    labels=False
)


# === Order by area (smallest on top) ===
# q_gpkg = q_gpkg.sort_values(by='area', ascending=True)
q_gpkg = q_gpkg.sort_values(by='area', ascending=False)
gwl_gpkg = gwl_gpkg.sort_values(by='gw_memory', ascending=False)

# === Define the bounding box (Chile region) ===
min_lon, max_lon = -74, -69
min_lat, max_lat = -41.8, -27
gwl_gpkg = gwl_gpkg[(gwl_gpkg.geometry.y <= max_lat) & (gwl_gpkg.geometry.y >= min_lat)]
q_gpkg = q_gpkg.cx[min_lon:max_lon, min_lat:max_lat]
elevation = elevation.sel(longitude=slice(min_lon, max_lon), latitude=slice(max_lat, min_lat))

# === Define the figure and axes (side by side) ===
fig = plt.figure(figsize=(12, 8))

width = 0.4
height = 0.85
x1 = 0.05
x2 = x1 + width - 0.22
x3 = x2 + width - 0.18

# === Create axes with manual placement ===
ax1 = fig.add_axes([x1, 0.1, width, height])  # [left, bottom, width, height]
ax2 = fig.add_axes([x2, 0.1, width, height])  # right panel slightly shifted
# ax3 = fig.add_axes([x3, 0.1, width, height])  # right panel slightly shifted

size_pt = 50

# === Plot blue ocean background ===
ax1.set_facecolor(to_rgba('powderblue', 0.3))
ax2.set_facecolor(to_rgba('powderblue', 0.3))

# === Plot the NetCDF elevation data ===
elevation.plot(ax=ax1, cmap='Greys', alpha=1, add_colorbar=False)
elevation.plot(ax=ax2, cmap='Greys', alpha=1, add_colorbar=False)

regions_gdf.plot(ax=ax1, edgecolor='black', facecolor='none', linewidth=0.2, alpha=0.8)
regions_gdf.plot(ax=ax2, edgecolor='black', facecolor='none', linewidth=0.2, alpha=0.8)

q_gpkg.plot(ax=ax1, column='bin_idx', cmap=cmap, norm=plt.Normalize(vmin=0, vmax=len(labels)-1),
            alpha=1, edgecolor='black', linewidth=0.1, label='Streamflow Memory')


gwl_gpkg.plot(ax=ax2, column='bin_idx', cmap=cmap, norm=plt.Normalize(vmin=0, vmax=len(labels)-1),
              markersize=size_pt, alpha=1, edgecolor='black', linewidth=0.2, label='Groundwater Memory')

# === Add titles ===
ax1.set_title('a) Streamflow memory', fontsize = 10, loc='left')
ax2.set_title('b) Groundwater memory', fontsize = 10, loc='left')
ax2.set_yticklabels('')

# === Common colorbar ===
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, ax=[ax1, ax2], boundaries=boundaries,ticks=tick_positions, 
                    orientation='vertical', fraction=0.02, pad=0.015)

# === Set the labels correctly ===
cbar.ax.set_yticklabels(labels)
# cbar.set_label('Hydro Memory (months)')
cbar.ax.set_title("Memory (months)", fontsize = 10, loc='left', pad=10)

# === Remove axis labels and ticks ===
for ax in [ax1, ax2]:
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.grid(True, linestyle='--', alpha=0.5)

    # === Save the figure ===
plt.tight_layout()
plt.savefig('Fig_maps_GW_Q_memory.png', dpi=300, bbox_inches='tight')
plt.show()


/Users/cag/Dropbox/0_Research/0_githubs/ms_recovery/.venv/lib/python3.11/site-packages/pyproj/__init__.py:96: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_DATA (PROJ 9.1+) | PROJ_LIB (PROJ<9.1) or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


DataDirError: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_DATA (PROJ 9.1+) | PROJ_LIB (PROJ<9.1) or with `pyproj.datadir.set_data_dir`.